## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


확률적 경사 하강법, 미니배치 경사 하강법

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

- 학습률을 조절해, 알고리즘이 최적점에 도달하도록 한다. 반복 횟수를 크게 지정하고 그레이디언트 벡터가 작아지면, 즉 벡터의 노름이 어떤 값 ε 보다 작아지면 경사 하강법이 거의 최솟값에 도달한 것이므로 알고리즘을 중지한다.
- 조기 종료
- 릿지, 라쏘 등 규제 적용

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

과소적합이 발생한 상황으로, 높은 편향 문제로, 규제 하이퍼파라미터 𝛼를 줄여야 함.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

1. 규제 없는 선형 회귀 대신 릿지 회귀를 사용하는 이유는 모델의 가중치를 가능한 한 작게 유지하게 하기 위함.
2. 릿지 회귀 대신 라쏘 회귀를 사용하는 이유는 덜 중요한 가중치를 제거하기 위함.
3. 라쏘 회귀 대신 엘라스틱넷을 사용하는 이유는 릿지 회귀와 라쏘 회귀를 절충한 모델로, 혼합 비율 r을 사용해 조절하기 위함. (다중공선성에 더 강한 모습을 보임)

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [9]:
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
X = iris["data"][:, (2, 3)]
y = iris["target"]

# bias 추가
X_with_bias = np.c_[np.ones([len(X), 1]), X]

np.random.seed(2042)

test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

# 올바른 인덱스 분할
train_idx = rnd_indices[:train_size]
valid_idx = rnd_indices[train_size:train_size + validation_size]
test_idx = rnd_indices[train_size + validation_size:]

X_train = X_with_bias[train_idx]
y_train = y[train_idx]

X_valid = X_with_bias[valid_idx]
y_valid = y[valid_idx]

X_test = X_with_bias[test_idx]
y_test = y[test_idx]

# 소프트맥스 (수치적으로 안정화)
def softmax(logits):
    # logits: (n_samples, n_classes)
    shifted = logits - np.max(logits, axis=1, keepdims=True)   # 안정성 위해 최대값 빼기
    exps = np.exp(shifted)
    return exps / np.sum(exps, axis=1, keepdims=True)

# 원-핫 인코딩
def one_hot(y, n_classes):
    return np.eye(n_classes)[y]

# 하이퍼파라미터
eta = 0.1
n_iterations = 5000
epsilon = 1e-7
alpha = 0.1   # L2 규제 강도
best_loss = np.inf

n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))

Y_train_oh = one_hot(y_train, n_outputs)
Y_valid_oh = one_hot(y_valid, n_outputs)

# 파라미터 초기화
Theta = np.random.randn(n_inputs, n_outputs) * 0.01

# 조기 종료 설정 (patience 사용)
patience = 10
checks_without_progress = 0
best_theta = None

m = len(X_train)

for iteration in range(n_iterations):
    # 배치 경사 하강법 (전체 훈련 데이터 사용)
    logits = X_train.dot(Theta)            # (m, n_outputs)
    Y_proba = softmax(logits)              # (m, n_outputs)

    # gradient: (1/m) * X^T (Y_proba - Y_onehot) + reg
    error = Y_proba - Y_train_oh
    # 규제는 bias(첫 행)에 대해서는 적용하지 않음
    reg_term = (alpha / m) * np.vstack([np.zeros((1, n_outputs)), Theta[1:]])
    gradients = (1.0 / m) * X_train.T.dot(error) + reg_term

    Theta = Theta - eta * gradients

    # 검증 손실 계산 (교차 엔트로피 + L2)
    val_logits = X_valid.dot(Theta)
    val_proba = softmax(val_logits)
    xentropy_loss = -np.mean(np.sum(Y_valid_oh * np.log(val_proba + epsilon), axis=1))
    l2_loss = 0.5 * np.sum(np.square(Theta[1:]))   # bias 제외
    loss = xentropy_loss + alpha * l2_loss

    if iteration % 100 == 0:
        print(f"iter {iteration:4d}  valid loss = {loss:.6f}")

    # 조기 종료 체크
    if loss < best_loss - 1e-8:
        best_loss = loss
        best_theta = Theta.copy()
        checks_without_progress = 0
    else:
        checks_without_progress += 1
        if checks_without_progress >= patience:
            print(f"조기 종료: iter={iteration}, best_valid_loss={best_loss:.6f}")
            Theta = best_theta  # 검증 성능이 가장 좋았던 파라미터로 복원
            break

# 최종 평가 (테스트 정확도)
test_logits = X_test.dot(Theta)
test_proba = softmax(test_logits)
y_pred = np.argmax(test_proba, axis=1)
test_acc = np.mean(y_pred == y_test)
print("Test accuracy:", test_acc)

iter    0  valid loss = 1.017289
iter  100  valid loss = 0.699565
iter  200  valid loss = 0.634423
조기 종료: iter=285, best_valid_loss=0.626053
Test accuracy: 0.9333333333333333
